In [0]:
import pandas as pd
import numpy as np
import pickle

class Tree_node():
  def __init__(self, ids = None, child_node = [], entropy = 0, depth = 0):
    self.ids = ids #chi so node chua
    self.child_nodes = [] # danh sach cac node con
    self.entropy = entropy  #entropy muc nhieu loan cua node
    self.depth = depth #do sau cua node
    self.split_attribute = None #thuoc tinh de chia child node
    self.split_value = None #gia tri cu thuoc tinh can chia
    self.label = None #nhan cua node neu la leaf
  def set_propertie(self, split_attribute, split_value):
    self.split_attribute = split_attribute
    self.split_value = split_value
  def set_label(self, label):
    self.label = label
  



In [0]:
class Decision_Tree():
    def __init__(self, max_depth = 10, min_samples_split = 2, min_gain = 1e-4):
      self.root = None
      self.max_depth = max_depth #do sau lon nhat cua cay
      self.min_samples_split = min_samples_split  #so luong mau be nhat co the chia
      self.min_gain = min_gain  # do giam be nhat, neu nhor hon khong chia nodfree
      self.n_train = 0  # so luong diem du lieu

    #Tinh entropy  
    def _entropy(self, ids):
      if ids.shape[0] == 0: return 0
      arr, fred = np.unique(self.target[ids], return_counts = True) # tinh tan suat xuat hien cua cac class trong node
      fred = fred / fred.sum() # tinh xac xuat
      return -np.sum(fred*np.log(fred)) #entropy

    #set label cho diem du lieu
    def _set_label(self, node):
      arr, fred = np.unique(self.target[node.ids], return_counts = True)
      node.set_label(arr[np.argmax(fred)])
    
    #chia day ids
    def split_ids(self, ids, col, value):
      ids_node1 = ids[self.data[ids, col] < value] # ids_node1 = chi so cac phan tu trong node co gia tri cua attribute col <value 
      ids_node2 = ids[self.data[ids, col] >= value] #ids_node2 = chi so cac phan tu trong node co gia tri cua attribute col  >=value
      return ids_node1, ids_node2

    #chia node thanh child_nodes
    def split(self, node): # chia node thanh child node
      ids = node.ids
      best_gain = 0 #do giam tot nhat khi chia  attribute
      best_attribute = None # thuoc tinh tot nhat de chia
      value_split = None #gia tri cua thuoc tinh(chia theo < value_splits vaf >= value_splits)
      node_data = self.data[ids, :] #du lieu cua node
      cols = self.data.shape[1] # clo laf so thuoc tinh cua du lieu
      best_splits = []  #tap cac chi so chia thanh cac node
      for col in range(cols):
        unique_values = np.unique(node_data[:, col]) #tim cac gia tri cua attribute moi gia tri lay 1 lan va sap xep tang dan 
        for value in unique_values:
          ids_node1, ids_node2 = self.split_ids(ids, col, value)
          splits = np.array([ids_node1, ids_node2])
          p = ids_node1.shape[0]/ids.shape[0]
          gain = node.entropy - p*self._entropy(ids_node1)-(1-p)*self._entropy(ids_node2)
          if gain < self.min_gain: continue
          if gain > best_gain:
            best_gain = gain
            best_attribute = col
            best_splits = splits
            value_split = value
      node.set_propertie(best_attribute, value_split)
      child_nodes = [Tree_node(ids = split, entropy = self._entropy(split), depth = node.depth+1) for split in best_splits]
      return child_nodes
    

    #Tim decision tree
    def fit(self, data, target):
        self.n_train = data.shape[0]
        self.data = data
        self.target = target
        ids = np.array(range(self.n_train))
        self.root = Tree_node(ids = ids, entropy = self._entropy(ids), depth = 0)
        queue = [self.root]
        while queue:
          node = queue.pop()
          if node.depth < self.max_depth or node.entropy < self.min_gain:
            node.child_nodes = self.split(node)
            if not node.child_nodes:
              self._set_label(node)
            queue += node.child_nodes
          else:
            self._set_label(node)
    

    #du doan label
    def predict(self, data):
      n_data = data.shape[0]
      labels = [None]*n_data
      for i in range(n_data):
        x = data[i, :]
        node = self.root
        while node.child_nodes:
          value = x[node.split_attribute]
          if value < node.split_value:
            node = node.child_nodes[0]
          else:
            node = node.child_nodes[1]
        labels[i] = node.label
      return labels




In [0]:
n_inputs = 4
n_hours = 12
size_train = 2*250
file_path_old = "../content/drive/My Drive/VDK/data/dananghour2019.csv"
file_path_new = []
file_path_new.append("../content/drive/My Drive/VDK/data/danangdaily.csv")
file_path_new.append("../content/drive/My Drive/VDK/data/danangdaily1.csv")
file_path_new.append( "../content/drive/My Drive/VDK/data/danangdaily2.csv")
file_path_new.append( "../content/drive/My Drive/VDK/data/danangdaily3.csv")
def read_data(file_path):
  data=pd.read_csv(file_path, delimiter=',',header=11,skipinitialspace=True, usecols=['Temperature', 'Relative Humidity', 'Mean Sea Level Pressure', 'Total Precipitation'] )
  X = np.array(data)
  # X[X[:, n_inputs - 1 ]/2 >= 0.3, n_inputs - 1] = 1
  # X[X[:, n_inputs - 1 ]/2 < 0.3, n_inputs - 1] = 0
  return X
data = read_data(file_path_old)
data = data.reshape(-1, n_hours, n_inputs)
print(data.shape)
X = np.empty((0, n_inputs))
for i in range(data.shape[0]):
  t = data[i].mean(axis=0, dtype=float).reshape(-1, 4)
  t[t[:, n_inputs - 1] >= 0.3, n_inputs - 1] = 1
  t[t[:, n_inputs - 1] < 0.3, n_inputs - 1 ] = 0
  X = np.concatenate((X, t), axis = 0) 

y = X[1:,n_inputs - 1]
X = X[:-1]

X_train = X[:size_train]
y_train = y[:size_train]

X_test = X[size_train:]
y_test = y[size_train:]

# X_test = np.empty((0,n_inputs))               
# y_test = np.empty((0))
# for path in file_path_new:
#   data = read_data(path)
#   target = data[1:, n_inputs - 1]
#   data = data[:-1]
#   X_test = np.concatenate((X_test, data), axis = 0)
#   y_test = np.concatenate((y_test, target)) 
# temperature = np.array(data["Temperature"]).reshape(-1, 1)
# relative_humidity = np.array(data["Relative Humidity"]).reshape(-1, 1)
# MSLP = np.array(data["Mean Sea Level Pressure"]).reshape(-1, 1)
# precipitation_total = np.array(data["Precipitation Total"]).reshape(-1, 1)
# X = np.concatenate((temperature, relative_humidity, MSLP, precipitation_total), axis = 1)


print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train)


(730, 12, 4)
(500, 4) (500,) (229, 4) (229,)
[[2.05597326e+01 8.67283888e+01 1.02193333e+03 0.00000000e+00]
 [2.04797326e+01 8.74960220e+01 1.02090000e+03 0.00000000e+00]
 [2.00722329e+01 8.94948286e+01 1.02099167e+03 0.00000000e+00]
 ...
 [3.06047331e+01 8.45675884e+01 1.00375833e+03 1.00000000e+00]
 [2.93097336e+01 8.35675106e+01 1.00475833e+03 0.00000000e+00]
 [3.00422333e+01 8.22056734e+01 1.00531667e+03 0.00000000e+00]]


In [0]:
for i in range(20):
  tree = Decision_Tree(max_depth=i+1, min_samples_split=2)
  tree.fit(X_train, y_train)
  s = "../content/drive/My Drive/VDK/model/tree"+str(i+1)+".pkl"
  print(s)
  with open(s, "wb") as f:
    pickle.dump(tree, f)
    f.close()
  # with open(s, "rb") as f:
  #     p = pickle.load(f)
  # print(100*np.mean(y_train == p.predict(X_train)))
  # print(100*np.mean(y_test == p.predict(X_test)))


../content/drive/My Drive/VDK/model/tree1.pkl
../content/drive/My Drive/VDK/model/tree2.pkl
../content/drive/My Drive/VDK/model/tree3.pkl
../content/drive/My Drive/VDK/model/tree4.pkl
../content/drive/My Drive/VDK/model/tree5.pkl
../content/drive/My Drive/VDK/model/tree6.pkl
../content/drive/My Drive/VDK/model/tree7.pkl
../content/drive/My Drive/VDK/model/tree8.pkl
../content/drive/My Drive/VDK/model/tree9.pkl
../content/drive/My Drive/VDK/model/tree10.pkl
../content/drive/My Drive/VDK/model/tree11.pkl
../content/drive/My Drive/VDK/model/tree12.pkl
../content/drive/My Drive/VDK/model/tree13.pkl
../content/drive/My Drive/VDK/model/tree14.pkl
../content/drive/My Drive/VDK/model/tree15.pkl
../content/drive/My Drive/VDK/model/tree16.pkl
../content/drive/My Drive/VDK/model/tree17.pkl
../content/drive/My Drive/VDK/model/tree18.pkl
../content/drive/My Drive/VDK/model/tree19.pkl
../content/drive/My Drive/VDK/model/tree20.pkl
